In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import importlib
%matplotlib inline

In [2]:
#!git clone https://github.com/ieee8023/covid-chestxray-dataset.git
#!wget https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip

In [3]:
import requests

In [4]:

libs = [trandata_tranformform, dataset , Net]

In [5]:
for lib in libs:
    data = requests.get(lib, allow_redirects=True).content
    open(lib.split("/")[-1], 'wb').write(data)

In [6]:
import dataset
import data_tranform
import Net

In [ ]:
importlib.reload(dataset)

In [ ]:
importlib.reload(data_tranform)

In [ ]:
importlib.reload(Net)